In [1]:
# Copyright Salvatore Di Carlo (WSU,LAL,CERN) 2017-2020
# https://github.com/sdicarlo/beamstrahlung_simulation

In [2]:
import pandas as pd
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from scipy.integrate import quad, dblquad
import random
import dill
from scipy.spatial.transform import Rotation as Rotation
from scipy import interpolate

In [3]:
#Number of simulations
n_macro = 100
n_steps = 100

#Beams Parameters SuperKEKB
theta_c = 83.0e-03
C = 3016
n_b = 2500
E1 = 4.0e+09
E2 = 7.0e+09

#Target beam is 1
sigma1_X = 10.0e-06
sigma1_Y = 4.8e-08
sigma1_Z = 6.0e-03
beta1_X = 0.032
beta1_Y = 2.7e-04
N1 = 9.04e+10

#Radiating beam is 2
sigma2_X = 11.0e-06
sigma2_Y = 6.2e-08
sigma2_Z = 5.0e-03
beta2_X = 0.025
beta2_Y = 3.0e-04
N2 = 6.53e+10

#Time interval for simulation
T1 = -5e-11
T2 = +5e-11

#Beams Parameters LEP for benchmark with Bassetti
# theta_c = 0.0e-03
# E1 = 51.5e+09
# E2 = 51.5e+09
# sigma1_X = 0.3e-03
# sigma1_Y = 0.019e-03
# sigma1_Z = 12.8e-03
# N1 = 4.2e+11
# N2 = 4.2e+11
# T1 = -1.0e-10
# T2 = +1.0e-10

#Fundamental constants
eps_0 = 8.854187817e-12
pi = np.pi
e = 1.6021766208e-19
m1 = 1.67262192369e-27
m2 = 9.10938356e-31
c = 299792458.0
h_planck = 6.62607015e-34

#collision frequency
f_c=n_b*c/C
Lum_0 = 1e-4*f_c*N1*N2/(2*pi*(np.sqrt(sigma1_Z**2+sigma2_Z**2)*theta_c/2)*np.sqrt(sigma1_Y**2+sigma2_Y**2))

#Conversion units
jouleToKev = 6.242e+15

#Position of the mirror (to be checked)
theta_u_min = 8.32e-03
theta_u_max = 8.76e-03
theta_mirror_u = (theta_u_min + theta_u_max)/2.0
A_mirror = (2.0e-03)*(2.8e-03)
L_u = 4.57
h_u = L_u * np.tan(theta_mirror_u)

theta_d_min = 8.43e-03
theta_d_max = 8.87e-03
theta_mirror_d = (theta_d_min + theta_d_max)/2.0
A_mirror = (2.0e-03)*(2.8e-03)
L_d = 4.51
h_d = L_d * np.tan(theta_mirror_d)

#Energies in eV and calculation of gamma factors for LER and HER
E_e_rest = 0.5109989461e+06
E_p_rest = 938.27208816e+06
gamma1 = E1/E_e_rest
gamma2 = E2/E_e_rest
v1 = c * np.sqrt(1-(1/(gamma1*gamma1)))
v2 = c * np.sqrt(1-(1/(gamma2*gamma2)))

#Integration parameters
dt = float((T2-T1)/n_steps)
int_tolerance = 1.0e-8
q2_max = 0.01*0.01

#Initial position and velocity of the electron
r2_i_vec = np.array([v2 * abs(T1) * np.sin(theta_c), 0.0, v2 * abs(T1) * np.cos(theta_c)])
v2_vec = np.array([-v2 * np.sin(theta_c), 0.0, -v2 * np.cos(theta_c)])

#Defining a coordinate system with z in the direction of the electron trajectory
x_axis_vec = np.array([1,0,0]) # x versor
y_axis_vec = np.array([0,1,0]) # y versor
e_z_axis_vec = v2_vec/np.sqrt(v2_vec.dot(v2_vec)) # versor along electron propagation
e_y_axis_vec = y_axis_vec
e_x_axis_vec = (np.cross(y_axis_vec,e_z_axis_vec))/np.sqrt(np.cross(y_axis_vec,e_z_axis_vec).dot(np.cross(y_axis_vec,e_z_axis_vec))) # x is (y X z)

#The position of the mirror is defined in the electron frame
r_mirror_u_vec = L_u * e_z_axis_vec + h_u * e_y_axis_vec
n_mirror_u_vec = (e_z_axis_vec + e_y_axis_vec)/np.sqrt((e_z_axis_vec + e_y_axis_vec).dot((e_z_axis_vec + e_y_axis_vec)))

r_mirror_d_vec = L_d * e_z_axis_vec - h_d * e_y_axis_vec
n_mirror_d_vec = (e_z_axis_vec - e_y_axis_vec)/np.sqrt((e_z_axis_vec - e_y_axis_vec).dot((e_z_axis_vec - e_y_axis_vec)))

#minimum and maximum frequencies for visible range
f_vis_min = 430e12
f_vis_max = 770e12

In [4]:
def Luminosity(offsetX1,offsetX2,offsetY1,offsetY2,offsetZ1,offsetZ2,sigmaX1,sigmaY1,sigmaZ1,betaX1,betaY1,sigmaX2,sigmaY2,sigmaZ2,betaX2,betaY2):
   
    
    def x1(x,z):
        return x*np.cos(theta_c/2.0)-z*np.sin(theta_c/2.0)-offsetX1
    
    def x2(x,z):
        return -x*np.cos(theta_c/2.0)-z*np.sin(theta_c/2.0)-offsetX2
    
    def z1(x,z):
        return x*np.sin(theta_c/2.0)+z*np.cos(theta_c/2.0)-offsetZ1
    
    def z2(x,z):
        return x*np.sin(theta_c/2.0)-z*np.cos(theta_c/2.0)-offsetZ2
        
    def sigmaX1_z(x,z):
        return sigmaX1*np.sqrt(1.0+(z1(x,z)/betaX1)**2)
    
    def sigmaX2_z(x,z):
        return sigmaX2*np.sqrt(1.0+(z2(x,z)/betaX2)**2)
    
    def sigmaY1_z(x,z):
        return sigmaY1*np.sqrt(1.0+(z1(x,z)/betaY1)**2)
    
    def sigmaY2_z(x,z):
        return sigmaY2*np.sqrt(1.0+(z2(x,z)/betaY2)**2)

    def n0(x,z):
        return 1.0/(sigmaX1_z(x,z)*sigmaX2_z(x,z)*np.sqrt(sigmaY1_z(x,z)**2+sigmaY2_z(x,z)**2)*np.sqrt(sigmaZ1**2+sigmaZ2**2))
        
    def n1(x,z):
        return np.exp(-(z1(x,z)-z2(x,z))**2/(2.0*(sigmaZ1**2+sigmaZ2**2)))

    def n2(x,z):
        return np.exp(-(x1(x,z)**2)/(2.0*sigmaX1_z(x,z)**2)-(x2(x,z)**2)/(2.0*sigmaX2_z(x,z)**2))

    def n3(x,z):
        return np.exp(-(offsetY1-offsetY2)**2/(2.0*(sigmaY1_z(x,z)**2 + sigmaY2_z(x,z)**2)))

    def luminosityIntegrand(x,z):
        return 1e-4*2.0 * f_c * N1 * N2 * np.cos(theta_c/2.0)**2 * (1.0/(2.0*pi))**2*n0(x,z)*n1(x,z)*n2(x,z)*n3(x,z)

    return dblquad(luminosityIntegrand,-30.0e-03,30.0e-03,-200.0e-06,200.0e-06, args=(), epsabs=1.49e-08, epsrel=1.49e-08)

In [5]:
Luminosity(0,0,0,0,0,0,sigma1_X,sigma1_Y,sigma1_Z,beta1_X,beta1_Y,sigma2_X,sigma2_Y,sigma2_Z,beta2_X,beta2_Y)

(8.087025668761242e+35, 1.2464461976463533e+30)

In [6]:
def f_x(x,X,Y,Z,a,b):
    return np.exp(-(X*X/(a*a+x))-(Y*Y/(b*b+x)))/((a*a+x)*np.sqrt((a*a+x)*(b*b+x)))

def E_x(X,Y,Z,T,a,b,d):
    res1, err1 = quad(f_x, 0.0, 3.0*b*b, args=(X,Y,Z,a,b))
    res2, err2 = quad(f_x, 3.0*b*b, 3.0*a*a, args=(X,Y,Z,a,b))
    res3, err3 = quad(f_x, 3.0*a*a, q2_max, args=(X,Y,Z,a,b))
    res = res1+res2+res3
    return (1.0/(4.0*pi*eps_0)) * (2.0 * e * N1 / np.sqrt(pi)) * (X/d) * np.exp(-(((Z-v1*T)*(Z-v1*T))/(d*d))) * res

def f_y(x,X,Y,Z,a,b):
    return np.exp(-(X*X/(a*a+x))-(Y*Y/(b*b+x)))/((b*b+x)*np.sqrt((a*a+x)*(b*b+x)))

def E_y(X,Y,Z,T,a,b,d):
    res1, err1 = quad(f_y, 0.0, 3.0*b*b, args=(X,Y,Z,a,b))
    res2, err2 = quad(f_y, 3.0*b*b, 3.0*a*a, args=(X,Y,Z,a,b))
    res3, err3 = quad(f_y, 3.0*a*a, q2_max, args=(X,Y,Z,a,b))
    res = res1+res2+res3
    return (1.0/(4.0*pi*eps_0)) * (2.0 * e * N1 / np.sqrt(pi)) * (Y/d) * np.exp(-(((Z-v1*T)*(Z-v1*T))/(d*d))) * res

def f_z(x,X,Y,Z,a,b,d):
    return np.exp(-(X*X/(a*a+x))-(Y*Y/(b*b+x)))/((gamma1*d*gamma1*d+x)*np.sqrt((a*a+x)*(b*b+x)))

def E_z(X,Y,Z,T,a,b,d):
    res1, err1 = quad(f_z, 0.0, 3.0*b*b, args=(X,Y,Z,a,b,d))
    res2, err2 = quad(f_z, 3.0*b*b, 3.0*a*a, args=(X,Y,Z,a,b,d))
    res3, err3 = quad(f_z, 3.0*a*a, q2_max, args=(X,Y,Z,a,b,d))
    res = res1+res2+res3
    return (1.0/(4.0*pi*eps_0)) * (2.0 * e * N1 / np.sqrt(pi)) * ((Z-v1*T)/d) * np.exp(-(((Z-v1*T)*(Z-v1*T))/(d*d))) * res

In [7]:
def bs_vis_rates(rel_delta_x,rel_delta_y,rel_delta_z,
                 rel_delta_sigma1_x,rel_delta_sigma1_y,rel_delta_sigma1_z,
                 rel_delta_sigma2_x,rel_delta_sigma2_y,rel_delta_sigma2_z,
                 delta_phi1,delta_phi2):
    U_tot = 0.0
    U_x = 0.0
    U_y = 0.0
    U_z = 0.0

    U_mirror_u_x = 0.0
    U_mirror_u_y = 0.0
    U_mirror_u_z = 0.0
    U_mirror_u_tot = 0.0

    U_mirror_u_x_fft = 0.0
    U_mirror_u_y_fft = 0.0
    U_mirror_u_z_fft = 0.0

    U_mirror_u_x_vis = 0.0
    U_mirror_u_y_vis = 0.0
    U_mirror_u_z_vis = 0.0

    photon_mirror_u_x_vis = 0.0
    photon_mirror_u_y_vis = 0.0
    photon_mirror_u_z_vis = 0.0

    U_mirror_d_x = 0.0
    U_mirror_d_y = 0.0
    U_mirror_d_z = 0.0
    U_mirror_d_tot = 0.0

    U_mirror_d_x_fft = 0.0
    U_mirror_d_y_fft = 0.0
    U_mirror_d_z_fft = 0.0

    U_mirror_d_x_vis = 0.0
    U_mirror_d_y_vis = 0.0
    U_mirror_d_z_vis = 0.0

    photon_mirror_d_x_vis = 0.0
    photon_mirror_d_y_vis = 0.0
    photon_mirror_d_z_vis = 0.0  


    for i in range(n_macro):
        #Restoring initial conditions

        t_mirror_u_arr = []
        p_mirror_u_x_arr = []
        p_mirror_u_y_arr = []
        p_mirror_u_z_arr = []

        t_mirror_d_arr = []
        p_mirror_d_x_arr = []
        p_mirror_d_y_arr = []
        p_mirror_d_z_arr = []

        t = T1
        r2_i_vec = np.array([v2 * abs(T1) * np.sin(theta_c), 0.0, v2 * abs(T1) * np.cos(theta_c)])
        v2_vec = np.array([-v2 * np.sin(theta_c), 0.0, -v2 * np.cos(theta_c)])
        r_mirror_u_vec = L_u * e_z_axis_vec + h_u * e_y_axis_vec
        n_mirror_u_vec = (e_z_axis_vec + e_y_axis_vec)/np.sqrt((e_z_axis_vec + e_y_axis_vec).dot((e_z_axis_vec + e_y_axis_vec)))
        r_mirror_d_vec = L_d * e_z_axis_vec - h_d * e_y_axis_vec
        n_mirror_d_vec = (e_z_axis_vec - e_y_axis_vec)/np.sqrt((e_z_axis_vec - e_y_axis_vec).dot((e_z_axis_vec - e_y_axis_vec)))

        #Shifting the electron from the center of the beam in a random way
        delta_x = random.gauss(rel_delta_x*sigma2_X,rel_delta_sigma2_x*sigma2_X)
        delta_y = random.gauss(rel_delta_y*sigma2_Y,rel_delta_sigma2_y*sigma2_Y)
        delta_z = random.gauss(rel_delta_z*sigma2_Z,rel_delta_sigma2_z*sigma2_Z)

        delta_r2_i_vec = delta_x * e_x_axis_vec + delta_y * e_y_axis_vec + delta_z * e_z_axis_vec
        r2_i_vec += delta_r2_i_vec
        
        r2_i_vec = Rotation.from_rotvec(delta_phi1*1e-3*x_axis_vec).apply(r2_i_vec)
        v2_vec = Rotation.from_rotvec(delta_phi1*1e-3*x_axis_vec).apply(v2_vec)
        r_mirror_u_vec = Rotation.from_rotvec(delta_phi1*1e-3*x_axis_vec).apply(r_mirror_u_vec)
        n_mirror_u_vec = Rotation.from_rotvec(delta_phi1*1e-3*x_axis_vec).apply(n_mirror_u_vec)
        r_mirror_d_vec = Rotation.from_rotvec(delta_phi1*1e-3*x_axis_vec).apply(r_mirror_d_vec)
        n_mirror_d_vec = Rotation.from_rotvec(delta_phi1*1e-3*x_axis_vec).apply(n_mirror_d_vec)
        
        r2_i_vec = Rotation.from_rotvec(delta_phi2*1e-3*e_x_axis_vec).apply(r2_i_vec)
        v2_vec = Rotation.from_rotvec(delta_phi2*1e-3*e_x_axis_vec).apply(v2_vec)
        
        #The electron in its initial position
        x = r2_i_vec[0]
        y = r2_i_vec[1]
        z = r2_i_vec[2]

        for k in range(n_steps):
            r2_vec = r2_i_vec + v2_vec * (t-T1) 
            x = r2_vec[0]
            y = r2_vec[1]
            z = r2_vec[2]      

            #The electric field due to the beam is calculated at the electron position (x,y,z,t)
            E21_x = E_x(x,y,z,t,rel_delta_sigma1_x*np.sqrt(2.0)*sigma1_X, rel_delta_sigma1_y*np.sqrt(2.0)*sigma1_Y, rel_delta_sigma1_z*np.sqrt(2.0)*sigma1_Z)
            E21_y = E_y(x,y,z,t,rel_delta_sigma1_x*np.sqrt(2.0)*sigma1_X, rel_delta_sigma1_y*np.sqrt(2.0)*sigma1_Y, rel_delta_sigma1_z*np.sqrt(2.0)*sigma1_Z)
            E21_z = E_z(x,y,z,t,rel_delta_sigma1_x*np.sqrt(2.0)*sigma1_X, rel_delta_sigma1_y*np.sqrt(2.0)*sigma1_Y, rel_delta_sigma1_z*np.sqrt(2.0)*sigma1_Z)

            #Defining variables for radiation Field (see Jackson pag.664)
            R_u = r_mirror_u_vec - r2_vec
            R_d = r_mirror_d_vec - r2_vec

            beta2_vec = v2_vec * (1.0/c)
            n_u = R_u/np.sqrt(R_u.dot(R_u))
            n_d = R_d/np.sqrt(R_d.dot(R_d))


            #Calculating the retarded time from the emission position
            t_ret_u = np.sqrt(R_u.dot(R_u))/c;
            t_ret_d = np.sqrt(R_d.dot(R_d))/c;

            #Setting electric and magnetic fields due to the beam and calculated at the electron position (x,y,z)
            E21_vec = np.array([E21_x,E21_y,E21_z])
            B21_vec = np.array([-(v1/(c*c))*E21_y, (v1/(c*c))*E21_x, 0.0])
            F21_vec = -e * (E21_vec + np.cross(v2_vec,B21_vec))

            a2_vec = (1.0/(m2*gamma2)) * (F21_vec - ((v2_vec.dot(F21_vec) * (1.0/(c*c))) * v2_vec))

            a2_e_x_axis_vec = np.array([a2_vec.dot(e_x_axis_vec),0.0,0.0])
            a2_e_y_axis_vec = np.array([0.0,a2_vec.dot(e_y_axis_vec),0.0])
            a2_e_z_axis_vec = np.array([0.0,0.0,a2_vec.dot(e_z_axis_vec)])

            P_x = (e*e/(6.0*pi*eps_0*c*c*c)) * gamma2 * gamma2 * gamma2 * gamma2 * a2_e_x_axis_vec.dot(a2_e_x_axis_vec)
            P_y = (e*e/(6.0*pi*eps_0*c*c*c)) * gamma2 * gamma2 * gamma2 * gamma2 * a2_e_y_axis_vec.dot(a2_e_y_axis_vec)
            P_z = (e*e/(6.0*pi*eps_0*c*c*c)) * gamma2 * gamma2 * gamma2 * gamma2 * gamma2 * gamma2 * a2_e_z_axis_vec.dot(a2_e_z_axis_vec)       

            P_tot = P_x + P_y + P_z

            U_tot += P_tot*dt
            U_x += P_x*dt
            U_y += P_y*dt
            U_z += P_z*dt

            #Formula for the radiation Field from a moving point charge
            beta2_dot_vec = a2_vec * (1.0/c)

            E_rad_u = (-e/c) * (1.0/(4.0*pi*eps_0)) * (1.0/((1.0 - beta2_vec.dot(n_u))*(1.0 - beta2_vec.dot(n_u))*(1.0 - beta2_vec.dot(n_u))*np.sqrt(R_u.dot(R_u)))) * (((n_u-beta2_vec) * n_u.dot(beta2_dot_vec)) - ((beta2_dot_vec) * (n_u.dot(n_u-beta2_vec))))
            E_rad_d = (-e/c) * (1.0/(4.0*pi*eps_0)) * (1.0/((1.0 - beta2_vec.dot(n_d))*(1.0 - beta2_vec.dot(n_d))*(1.0 - beta2_vec.dot(n_d))*np.sqrt(R_d.dot(R_d)))) * (((n_d-beta2_vec) * n_d.dot(beta2_dot_vec)) - ((beta2_dot_vec) * (n_d.dot(n_d-beta2_vec))))

            E_rad_u_x = E_rad_u.dot(e_x_axis_vec)
            E_rad_u_y = E_rad_u.dot(e_y_axis_vec)
            E_rad_u_z = E_rad_u.dot(e_z_axis_vec)

            E_rad_d_x = E_rad_d.dot(e_x_axis_vec)
            E_rad_d_y = E_rad_d.dot(e_y_axis_vec)
            E_rad_d_z = E_rad_d.dot(e_z_axis_vec)


            #Power emitted in solid angle of mirror
            P_mirror_u_x = c * eps_0 * E_rad_u.dot(e_x_axis_vec) * E_rad_u.dot(e_x_axis_vec) * (n_u.dot(n_mirror_u_vec)) * A_mirror
            P_mirror_u_y = c * eps_0 * E_rad_u.dot(e_y_axis_vec) * E_rad_u.dot(e_y_axis_vec) * (n_u.dot(n_mirror_u_vec)) * A_mirror
            P_mirror_u_z = c * eps_0 * E_rad_u.dot(e_z_axis_vec) * E_rad_u.dot(e_z_axis_vec) * (n_u.dot(n_mirror_u_vec)) * A_mirror
            P_mirror_u_tot = c * eps_0 * E_rad_u.dot(E_rad_u) * (n_u.dot(n_mirror_u_vec)) * A_mirror

            P_mirror_d_x = c * eps_0 * E_rad_d.dot(e_x_axis_vec) * E_rad_d.dot(e_x_axis_vec) * (n_d.dot(n_mirror_d_vec)) * A_mirror
            P_mirror_d_y = c * eps_0 * E_rad_d.dot(e_y_axis_vec) * E_rad_d.dot(e_y_axis_vec) * (n_d.dot(n_mirror_d_vec)) * A_mirror
            P_mirror_d_z = c * eps_0 * E_rad_d.dot(e_z_axis_vec) * E_rad_d.dot(e_z_axis_vec) * (n_d.dot(n_mirror_d_vec)) * A_mirror
            P_mirror_d_tot = c * eps_0 * E_rad_d.dot(E_rad_d) * (n_d.dot(n_mirror_d_vec)) * A_mirror           

            p_mirror_u_x = np.sqrt(c * eps_0) * E_rad_u.dot(e_x_axis_vec) * np.sqrt((n_u.dot(n_mirror_u_vec)) * A_mirror)
            p_mirror_u_y = np.sqrt(c * eps_0) * E_rad_u.dot(e_y_axis_vec) * np.sqrt((n_u.dot(n_mirror_u_vec)) * A_mirror)
            p_mirror_u_z = np.sqrt(c * eps_0) * E_rad_u.dot(e_z_axis_vec) * np.sqrt((n_u.dot(n_mirror_u_vec)) * A_mirror)        

            p_mirror_d_x = np.sqrt(c * eps_0) * E_rad_d.dot(e_x_axis_vec) * np.sqrt((n_d.dot(n_mirror_d_vec)) * A_mirror)
            p_mirror_d_y = np.sqrt(c * eps_0) * E_rad_d.dot(e_y_axis_vec) * np.sqrt((n_d.dot(n_mirror_d_vec)) * A_mirror)
            p_mirror_d_z = np.sqrt(c * eps_0) * E_rad_d.dot(e_z_axis_vec) * np.sqrt((n_d.dot(n_mirror_d_vec)) * A_mirror) 

            U_mirror_u_x += P_mirror_u_x * dt * (1.0 - n_u.dot(beta2_vec))
            U_mirror_u_y += P_mirror_u_y * dt * (1.0 - n_u.dot(beta2_vec))
            U_mirror_u_z += P_mirror_u_z * dt * (1.0 - n_u.dot(beta2_vec))
            U_mirror_u_tot += P_mirror_u_tot * dt * (1.0 - n_u.dot(beta2_vec))

            U_mirror_d_x += P_mirror_d_x * dt * (1.0 - n_d.dot(beta2_vec))
            U_mirror_d_y += P_mirror_d_y * dt * (1.0 - n_d.dot(beta2_vec))
            U_mirror_d_z += P_mirror_d_z * dt * (1.0 - n_d.dot(beta2_vec))
            U_mirror_d_tot += P_mirror_d_tot * dt * (1.0 - n_d.dot(beta2_vec))

            t_mirror_u_arr.append(t + t_ret_u)
            p_mirror_u_x_arr.append(p_mirror_u_x)
            p_mirror_u_y_arr.append(p_mirror_u_y)
            p_mirror_u_z_arr.append(p_mirror_u_z)

            t_mirror_d_arr.append(t + t_ret_d)
            p_mirror_d_x_arr.append(p_mirror_d_x)
            p_mirror_d_y_arr.append(p_mirror_d_y)
            p_mirror_d_z_arr.append(p_mirror_d_z)

            t += dt


        inter_u_x_arr = interpolate.interp1d(t_mirror_u_arr, p_mirror_u_x_arr, kind='cubic')
        inter_u_y_arr = interpolate.interp1d(t_mirror_u_arr, p_mirror_u_y_arr, kind='cubic')
        inter_u_z_arr = interpolate.interp1d(t_mirror_u_arr, p_mirror_u_z_arr, kind='cubic')

        inter_d_x_arr = interpolate.interp1d(t_mirror_d_arr, p_mirror_d_x_arr, kind='cubic')
        inter_d_y_arr = interpolate.interp1d(t_mirror_d_arr, p_mirror_d_y_arr, kind='cubic')
        inter_d_z_arr = interpolate.interp1d(t_mirror_d_arr, p_mirror_d_z_arr, kind='cubic')

        t_mirror_u_arr = np.linspace(min(t_mirror_u_arr), max(t_mirror_u_arr), len(t_mirror_u_arr))
        p_mirror_u_x_arr = inter_u_x_arr(t_mirror_u_arr)
        p_mirror_u_y_arr = inter_u_y_arr(t_mirror_u_arr)
        p_mirror_u_z_arr = inter_u_z_arr(t_mirror_u_arr)

        t_mirror_d_arr = np.linspace(min(t_mirror_d_arr), max(t_mirror_d_arr), len(t_mirror_d_arr))
        p_mirror_d_x_arr = inter_d_x_arr(t_mirror_d_arr)
        p_mirror_d_y_arr = inter_d_y_arr(t_mirror_d_arr)
        p_mirror_d_z_arr = inter_d_z_arr(t_mirror_d_arr)

        p_mirror_u_x_fft_arr = np.fft.fft(p_mirror_u_x_arr)
        p_mirror_u_y_fft_arr = np.fft.fft(p_mirror_u_y_arr)
        p_mirror_u_z_fft_arr = np.fft.fft(p_mirror_u_z_arr)

        p_mirror_d_x_fft_arr = np.fft.fft(p_mirror_d_x_arr)
        p_mirror_d_y_fft_arr = np.fft.fft(p_mirror_d_y_arr)
        p_mirror_d_z_fft_arr = np.fft.fft(p_mirror_d_z_arr)

        delta_t_u = t_mirror_u_arr[1]-t_mirror_u_arr[0]
        f_mirror_u_arr = np.fft.fftfreq(n_steps, d=delta_t_u)
        delta_f_u = f_mirror_u_arr[1]-f_mirror_u_arr[0]

        delta_t_d = t_mirror_d_arr[1]-t_mirror_d_arr[0]
        f_mirror_d_arr = np.fft.fftfreq(n_steps, d=delta_t_d)
        delta_f_d = f_mirror_d_arr[1]-f_mirror_d_arr[0]       

        P_mirror_u_x_fft_arr = np.square(abs(p_mirror_u_x_fft_arr))*(delta_t_u/(delta_f_u*n_steps))
        P_mirror_u_y_fft_arr = np.square(abs(p_mirror_u_y_fft_arr))*(delta_t_u/(delta_f_u*n_steps))
        P_mirror_u_z_fft_arr = np.square(abs(p_mirror_u_z_fft_arr))*(delta_t_u/(delta_f_u*n_steps))

        P_mirror_d_x_fft_arr = np.square(abs(p_mirror_d_x_fft_arr))*(delta_t_d/(delta_f_d*n_steps))
        P_mirror_d_y_fft_arr = np.square(abs(p_mirror_d_y_fft_arr))*(delta_t_d/(delta_f_d*n_steps))
        P_mirror_d_z_fft_arr = np.square(abs(p_mirror_d_z_fft_arr))*(delta_t_d/(delta_f_d*n_steps))

        ####################
        inter_u_x_arr = interpolate.interp1d(f_mirror_u_arr, P_mirror_u_x_fft_arr, kind='cubic')
        inter_u_y_arr = interpolate.interp1d(f_mirror_u_arr, P_mirror_u_y_fft_arr, kind='cubic')
        inter_u_z_arr = interpolate.interp1d(f_mirror_u_arr, P_mirror_u_z_fft_arr, kind='cubic')

        inter_d_x_arr = interpolate.interp1d(f_mirror_d_arr, P_mirror_d_x_fft_arr, kind='cubic')
        inter_d_y_arr = interpolate.interp1d(f_mirror_d_arr, P_mirror_d_y_fft_arr, kind='cubic')
        inter_d_z_arr = interpolate.interp1d(f_mirror_d_arr, P_mirror_d_z_fft_arr, kind='cubic')
        
        f_mirror_u_arr = np.linspace(min(f_mirror_u_arr), max(f_mirror_u_arr), 100*len(f_mirror_u_arr))
        P_mirror_u_x_fft_arr = inter_u_x_arr(f_mirror_u_arr)
        P_mirror_u_y_fft_arr = inter_u_y_arr(f_mirror_u_arr)
        P_mirror_u_z_fft_arr = inter_u_z_arr(f_mirror_u_arr)

        f_mirror_d_arr = np.linspace(min(f_mirror_d_arr), max(f_mirror_d_arr), 100*len(f_mirror_d_arr))
        P_mirror_d_x_fft_arr = inter_d_x_arr(f_mirror_d_arr)
        P_mirror_d_y_fft_arr = inter_d_y_arr(f_mirror_d_arr)
        P_mirror_d_z_fft_arr = inter_d_z_arr(f_mirror_d_arr)
        
        delta_f_u = f_mirror_u_arr[1]-f_mirror_u_arr[0]
        delta_f_d = f_mirror_d_arr[1]-f_mirror_d_arr[0] 
        ####################
        
        U_mirror_u_x_fft += np.sum(P_mirror_u_x_fft_arr)*delta_f_u
        U_mirror_u_y_fft += np.sum(P_mirror_u_y_fft_arr)*delta_f_u
        U_mirror_u_z_fft += np.sum(P_mirror_u_z_fft_arr)*delta_f_u

        U_mirror_d_x_fft += np.sum(P_mirror_d_x_fft_arr)*delta_f_d
        U_mirror_d_y_fft += np.sum(P_mirror_d_y_fft_arr)*delta_f_d
        U_mirror_d_z_fft += np.sum(P_mirror_d_z_fft_arr)*delta_f_d

        U_mirror_u_x_vis += 2.0*np.sum(P_mirror_u_x_fft_arr[(f_mirror_u_arr>f_vis_min) & (f_mirror_u_arr<f_vis_max)])*delta_f_u
        U_mirror_u_y_vis += 2.0*np.sum(P_mirror_u_y_fft_arr[(f_mirror_u_arr>f_vis_min) & (f_mirror_u_arr<f_vis_max)])*delta_f_u
        U_mirror_u_z_vis += 2.0*np.sum(P_mirror_u_z_fft_arr[(f_mirror_u_arr>f_vis_min) & (f_mirror_u_arr<f_vis_max)])*delta_f_u

        U_mirror_d_x_vis += 2.0*np.sum(P_mirror_d_x_fft_arr[(f_mirror_d_arr>f_vis_min) & (f_mirror_d_arr<f_vis_max)])*delta_f_d
        U_mirror_d_y_vis += 2.0*np.sum(P_mirror_d_y_fft_arr[(f_mirror_d_arr>f_vis_min) & (f_mirror_d_arr<f_vis_max)])*delta_f_d
        U_mirror_d_z_vis += 2.0*np.sum(P_mirror_d_z_fft_arr[(f_mirror_d_arr>f_vis_min) & (f_mirror_d_arr<f_vis_max)])*delta_f_d

        photon_mirror_u_x_vis += 2.0*np.sum(P_mirror_u_x_fft_arr[(f_mirror_u_arr>f_vis_min) & (f_mirror_u_arr<f_vis_max)]/(h_planck*f_mirror_u_arr[(f_mirror_u_arr>f_vis_min) & (f_mirror_u_arr<f_vis_max)]))*delta_f_u
        photon_mirror_u_y_vis += 2.0*np.sum(P_mirror_u_y_fft_arr[(f_mirror_u_arr>f_vis_min) & (f_mirror_u_arr<f_vis_max)]/(h_planck*f_mirror_u_arr[(f_mirror_u_arr>f_vis_min) & (f_mirror_u_arr<f_vis_max)]))*delta_f_u
        photon_mirror_u_z_vis += 2.0*np.sum(P_mirror_u_z_fft_arr[(f_mirror_u_arr>f_vis_min) & (f_mirror_u_arr<f_vis_max)]/(h_planck*f_mirror_u_arr[(f_mirror_u_arr>f_vis_min) & (f_mirror_u_arr<f_vis_max)]))*delta_f_u

        photon_mirror_d_x_vis += 2.0*np.sum(P_mirror_d_x_fft_arr[(f_mirror_d_arr>f_vis_min) & (f_mirror_d_arr<f_vis_max)]/(h_planck*f_mirror_d_arr[(f_mirror_d_arr>f_vis_min) & (f_mirror_d_arr<f_vis_max)]))*delta_f_d
        photon_mirror_d_y_vis += 2.0*np.sum(P_mirror_d_y_fft_arr[(f_mirror_d_arr>f_vis_min) & (f_mirror_d_arr<f_vis_max)]/(h_planck*f_mirror_d_arr[(f_mirror_d_arr>f_vis_min) & (f_mirror_d_arr<f_vis_max)]))*delta_f_d
        photon_mirror_d_z_vis += 2.0*np.sum(P_mirror_d_z_fft_arr[(f_mirror_d_arr>f_vis_min) & (f_mirror_d_arr<f_vis_max)]/(h_planck*f_mirror_d_arr[(f_mirror_d_arr>f_vis_min) & (f_mirror_d_arr<f_vis_max)]))*delta_f_d


    U_tot = (U_tot * N2) /n_macro
    U_x = (U_x * N2) /n_macro
    U_y = (U_y * N2) /n_macro
    U_z = (U_z * N2) /n_macro

    U_mirror_u_tot = (U_mirror_u_tot * N2) /n_macro
    U_mirror_u_x = (U_mirror_u_x * N2) /n_macro
    U_mirror_u_y = (U_mirror_u_y * N2) /n_macro
    U_mirror_u_z = (U_mirror_u_z * N2) /n_macro

    U_mirror_u_x_fft = (U_mirror_u_x_fft * N2) /n_macro
    U_mirror_u_y_fft = (U_mirror_u_y_fft * N2) /n_macro
    U_mirror_u_z_fft = (U_mirror_u_z_fft * N2) /n_macro

    U_mirror_u_x_vis = (U_mirror_u_x_vis * N2) /n_macro
    U_mirror_u_y_vis = (U_mirror_u_y_vis * N2) /n_macro
    U_mirror_u_z_vis = (U_mirror_u_z_vis * N2) /n_macro

    photon_mirror_u_x_vis = (photon_mirror_u_x_vis * N2) /n_macro
    photon_mirror_u_y_vis = (photon_mirror_u_y_vis * N2) /n_macro
    photon_mirror_u_z_vis = (photon_mirror_u_z_vis * N2) /n_macro

    U_mirror_d_tot = (U_mirror_d_tot * N2) /n_macro
    U_mirror_d_x = (U_mirror_d_x * N2) /n_macro
    U_mirror_d_y = (U_mirror_d_y * N2) /n_macro
    U_mirror_d_z = (U_mirror_d_z * N2) /n_macro

    U_mirror_d_x_fft = (U_mirror_d_x_fft * N2) /n_macro
    U_mirror_d_y_fft = (U_mirror_d_y_fft * N2) /n_macro
    U_mirror_d_z_fft = (U_mirror_d_z_fft * N2) /n_macro

    U_mirror_d_x_vis = (U_mirror_d_x_vis * N2) /n_macro
    U_mirror_d_y_vis = (U_mirror_d_y_vis * N2) /n_macro
    U_mirror_d_z_vis = (U_mirror_d_z_vis * N2) /n_macro

    photon_mirror_d_x_vis = (photon_mirror_d_x_vis * N2) /n_macro
    photon_mirror_d_y_vis = (photon_mirror_d_y_vis * N2) /n_macro
    photon_mirror_d_z_vis = (photon_mirror_d_z_vis * N2) /n_macro

    print("U_tot = ",U_tot)
    print("U_x = ",U_x)
    print("U_y = ",U_y)
    print("U_z = ",U_z)

    print("Average energy loss (KeV) = ",jouleToKev*U_tot/N2)

    print("U_mirror_u_tot = ",U_mirror_u_tot)
    print("U_mirror_u_x = ",U_mirror_u_x)
    print("U_mirror_u_y = ",U_mirror_u_y)
    print("U_mirror_u_z = ",U_mirror_u_z)

    print("U_mirror_u_x_fft = ",U_mirror_u_x_fft)
    print("U_mirror_u_y_fft = ",U_mirror_u_y_fft)
    print("U_mirror_u_z_fft = ",U_mirror_u_z_fft)

    print("U_mirror_u_x_vis = ",U_mirror_u_x_vis)
    print("U_mirror_u_y_vis = ",U_mirror_u_y_vis)
    print("U_mirror_u_z_vis = ",U_mirror_u_z_vis)

    print("photon_mirror_u_x_vis = ",photon_mirror_u_x_vis)
    print("photon_mirror_u_y_vis = ",photon_mirror_u_y_vis)
    print("photon_mirror_u_z_vis = ",photon_mirror_u_z_vis)

    print("U_mirror_d_tot = ",U_mirror_d_tot)
    print("U_mirror_d_x = ",U_mirror_d_x)
    print("U_mirror_d_y = ",U_mirror_d_y)
    print("U_mirror_d_z = ",U_mirror_d_z)

    print("U_mirror_d_x_fft = ",U_mirror_d_x_fft)
    print("U_mirror_d_y_fft = ",U_mirror_d_y_fft)
    print("U_mirror_d_z_fft = ",U_mirror_d_z_fft)

    print("U_mirror_d_x_vis = ",U_mirror_d_x_vis)
    print("U_mirror_d_y_vis = ",U_mirror_d_y_vis)
    print("U_mirror_d_z_vis = ",U_mirror_d_z_vis)

    print("photon_mirror_d_x_vis = ",photon_mirror_d_x_vis)
    print("photon_mirror_d_y_vis = ",photon_mirror_d_y_vis)
    print("photon_mirror_d_z_vis = ",photon_mirror_d_z_vis)

    print("Ratio energyUpMirror/totalEnergy = ",U_mirror_u_tot/U_tot)
    print("Ratio energyDownMirror/totalEnergy = ",U_mirror_d_tot/U_tot)
    
    return photon_mirror_u_x_vis,photon_mirror_u_y_vis,photon_mirror_u_z_vis,photon_mirror_d_x_vis,photon_mirror_d_y_vis,photon_mirror_d_z_vis

In [8]:
rates = bs_vis_rates(0,0,0,1,1,1,1,1,1,0,0)

U_tot =  2.7246290065854684e-05
U_x =  1.9210475858452557e-05
U_y =  8.035814207225555e-06
U_z =  1.765131718551912e-16
Average energy loss (KeV) =  2.6044616017008413
U_mirror_u_tot =  3.640635172507925e-16
U_mirror_u_x =  2.5671099024666433e-16
U_mirror_u_y =  1.0734469778579196e-16
U_mirror_u_z =  7.82921833765117e-21
U_mirror_u_x_fft =  2.452668760808449e-16
U_mirror_u_y_fft =  1.0287679808298255e-16
U_mirror_u_z_fft =  7.503348804530082e-21
U_mirror_u_x_vis =  7.567816680851858e-18
U_mirror_u_y_vis =  3.8963061473311255e-18
U_mirror_u_z_vis =  2.8417808195112997e-22
photon_mirror_u_x_vis =  19.217504998087513
photon_mirror_u_y_vis =  10.077925716335304
photon_mirror_u_z_vis =  0.0007350358948143751
U_mirror_d_tot =  3.4622318420983545e-16
U_mirror_d_x =  2.4413107266711677e-16
U_mirror_d_y =  1.0208447291757804e-16
U_mirror_d_z =  7.6386251381519e-21
U_mirror_d_x_fft =  2.330277539469354e-16
U_mirror_d_y_fft =  9.774934121090332e-17
U_mirror_d_z_fft =  7.3142458414667e-21
U_mirror